In [4]:
import tensorflow as tf
from tensorflow.keras import layers, models, losses, optimizers
from tensorflow.keras.preprocessing import image_dataset_from_directory, text, sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd

# Define tokenizer
tokenizer = text.Tokenizer(num_words=10000)  # Adjust as needed

# Custom Transformer Block
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim)]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training=False):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

# Frequency Domain Analysis
class FrequencyDomainSpectralAnalysis(layers.Layer):
    def __init__(self):
        super(FrequencyDomainSpectralAnalysis, self).__init__()
        
    def call(self, inputs):
        gray = tf.image.rgb_to_grayscale(inputs)
        freq = tf.signal.fft2d(tf.cast(gray, tf.complex64))
        freq_shifted = tf.signal.fftshift(freq)
        magnitude = tf.abs(freq_shifted)
        magnitude = tf.math.log1p(magnitude)
        magnitude = tf.image.per_image_standardization(magnitude)
        magnitude = tf.image.resize(magnitude, [inputs.shape[1], inputs.shape[2]])
        return magnitude

# Convolutional Transformer Encoder (CTE) Branch
class ConvolutionalTransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, num_layers, num_heads, ff_dim):
        super(ConvolutionalTransformerEncoder, self).__init__()
        self.embedding = layers.Conv2D(embed_dim, kernel_size=3, strides=1, padding='same')
        self.pooling = layers.GlobalAveragePooling2D()
        self.transformer_layers = [
            TransformerBlock(embed_dim, num_heads, ff_dim)
            for _ in range(num_layers)
        ]
        self.fc = layers.Dense(embed_dim)
    
    def call(self, x, training=False):
        x = self.embedding(x)
        x = self.pooling(x)
        x = tf.expand_dims(x, axis=1)
        for transformer_layer in self.transformer_layers:
            x = transformer_layer(x, training=training)
        x = tf.reduce_mean(x, axis=1)
        x = self.fc(x)
        return x

# Language Transformer-Based Encoder
class LanguageTransformerEncoder(layers.Layer):
    def __init__(self, vocab_size, embed_dim, num_layers, num_heads, ff_dim):
        super(LanguageTransformerEncoder, self).__init__()
        self.embedding = layers.Embedding(vocab_size, embed_dim)
        self.transformer_layers = [
            TransformerBlock(embed_dim, num_heads, ff_dim)
            for _ in range(num_layers)
        ]
        self.fc = layers.Dense(embed_dim)
    
    def call(self, x, training=False):
        x = self.embedding(x)
        for transformer_layer in self.transformer_layers:
            x = transformer_layer(x, training=training)
        x = tf.reduce_mean(x, axis=1)
        x = self.fc(x)
        return x

# Weighted Feature Embeddings Fusion
class WeightedFeatureFusion(layers.Layer):
    def __init__(self, embed_dim):
        super(WeightedFeatureFusion, self).__init__()
        self.weight = self.add_weight(shape=(embed_dim,), initializer='ones', trainable=True)
    
    def call(self, feature1, feature2):
        fused = feature1 * self.weight + feature2 * (1 - self.weight)
        return fused

# SVFT Decoder and Classification Head Layers
class SVFTDecoder(layers.Layer):
    def __init__(self, embed_dim, num_classes):
        super(SVFTDecoder, self).__init__()
        self.fc = layers.Dense(num_classes)
        #self.activation('sigmoid')
    
    def call(self, x, training=False):
        x = self.fc(x)
        #x = self.activation(x)
        return x

class DeepfakeDetectionModel(tf.keras.Model):
    def __init__(self, vocab_size, embed_dim, num_layers, num_heads, ff_dim, num_classes, **kwargs):
        super(DeepfakeDetectionModel, self).__init__(**kwargs)
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        self.num_layers = num_layers
        self.num_heads = num_heads
        self.ff_dim = ff_dim
        self.num_classes = num_classes

        self.image_model = ConvolutionalTransformerEncoder(embed_dim, num_layers, num_heads, ff_dim)
        self.text_model = LanguageTransformerEncoder(vocab_size, embed_dim, num_layers, num_heads, ff_dim)
        self.fusion = WeightedFeatureFusion(embed_dim)
        self.decoder = SVFTDecoder(embed_dim, num_classes)

    def call(self, inputs, training=False):
        images, texts = inputs
        img_features = self.image_model(images, training=training)
        text_features = self.text_model(texts, training=training)
        fused_features = self.fusion(img_features, text_features)
        output = self.decoder(fused_features, training=training)
        return output

    def get_config(self):
        config = super(DeepfakeDetectionModel, self).get_config()
        config.update({
            "vocab_size": self.vocab_size,
            "embed_dim": self.embed_dim,
            "num_layers": self.num_layers,
            "num_heads": self.num_heads,
            "ff_dim": self.ff_dim,
            "num_classes": self.num_classes,
        })
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)

# Training script
if __name__ == "__main__":
    def load_image_text_dataset(image_dir, metadata_file, batch_size=8, img_size=(224, 224), max_seq_length=50):
        # Load images from directory
        image_dataset = image_dataset_from_directory(
            image_dir,
            labels=None,
            image_size=img_size,
            batch_size=batch_size,
            shuffle=False  # Shuffle will be handled later
        )

        # Load metadata from CSV file
        metadata = pd.read_csv(metadata_file)

        # Extract texts and labels from metadata
        texts = metadata['Name'].tolist()
        labels = metadata['Label'].tolist()

        # Convert string labels to integers
        label_mapping = {'Real': 0, 'Fake': 1}
        int_labels = [label_mapping[label] for label in labels]

        # Tokenize and pad text sequences
        tokenizer.fit_on_texts(texts)
        sequences = tokenizer.texts_to_sequences(texts)
        text_tensor = pad_sequences(sequences, maxlen=max_seq_length)
        label_tensor = tf.constant(int_labels)

        # Create a dataset by zipping images, texts, and labels
        # Ensure that image_dataset has the same order as metadata
        image_iterator = image_dataset.as_numpy_iterator()
        def gen():
            for img_batch, label_batch in image_iterator:
                # Assuming labels are not used from image_dataset
                for img in img_batch:
                    yield (img, text_tensor.pop(0)), label_tensor.numpy().tolist().pop(0)

        dataset = tf.data.Dataset.from_generator(
            gen,
            output_signature=(
                (
                    tf.TensorSpec(shape=(img_size[0], img_size[1], 3), dtype=tf.float32),
                    tf.TensorSpec(shape=(max_seq_length,), dtype=tf.int32)
                ),
                tf.TensorSpec(shape=(), dtype=tf.int32)
            )
        )
        dataset = dataset.shuffle(buffer_size=1000).batch(batch_size).prefetch(tf.data.AUTOTUNE)
        return dataset

    # Load dataset
    image_directory = "C:\\Users\\Lord Sen\\ML\\DeepFake_det\\FF++_Faces_224\\FF++_Faces_224"  # Update as needed
    metadata_file = "C:\\Users\\Lord Sen\\ML\\metadata.csv"  # Update as needed
    train_dataset = load_image_text_dataset(image_directory, metadata_file)
    
    # Define model hyperparameters
    vocab_size = 10000
    embed_dim = 128
    num_layers = 2
    num_heads = 4
    ff_dim = 256
    num_classes = 2

    # Initialize the model
    model = DeepfakeDetectionModel(
        vocab_size=vocab_size, 
        embed_dim=embed_dim, 
        num_layers=num_layers, 
        num_heads=num_heads, 
        ff_dim=ff_dim, 
        num_classes=num_classes
    )

    # Build the model by passing a sample input
    sample_images = tf.random.uniform((1, 224, 224, 3))
    sample_texts = tf.random.uniform((1, 50), maxval=vocab_size, dtype=tf.int32)  # Fix: Specify maxval
    model((sample_images, sample_texts), training=True)

   # Compile the model
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), 
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
        metrics=['accuracy']
    )

    # Train the model
    #model.fit(train_dataset, epochs=10)

    # Save the model in the `.keras` format
    model.save('deepfake_detection_model.keras')

    # Define custom objects for loading the model
    custom_objects = {
       "DeepfakeDetectionModel": DeepfakeDetectionModel, 
       "TransformerBlock": TransformerBlock,
       "ConvolutionalTransformerEncoder": ConvolutionalTransformerEncoder,
       "LanguageTransformerEncoder": LanguageTransformerEncoder,
       "WeightedFeatureFusion": WeightedFeatureFusion,
       "SVFTDecoder": SVFTDecoder
   }

# Load the model
loaded_model = tf.keras.models.load_model('deepfake_detection_model.keras', custom_objects=custom_objects)


Found 400 files.


C:\Users\Lord Sen\anaconda3\Lib\site-packages\keras\src\ops\nn.py:545: UserWarning: You are using a softmax over axis 3 of a tensor of shape (1, 4, 1, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(
C:\Users\Lord Sen\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'adam', because it has 150 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [24]:
import tensorflow as tf
from tensorflow.keras import layers, models, losses, optimizers
from tensorflow.keras.preprocessing import image_dataset_from_directory, text, sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd

# Define tokenizer
tokenizer = text.Tokenizer(num_words=10000)  # Adjust as needed

# Custom Transformer Block
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim)]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training=False):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

# Frequency Domain Analysis
class FrequencyDomainSpectralAnalysis(layers.Layer):
    def __init__(self):
        super(FrequencyDomainSpectralAnalysis, self).__init__()
        
    def call(self, inputs):
        gray = tf.image.rgb_to_grayscale(inputs)
        freq = tf.signal.fft2d(tf.cast(gray, tf.complex64))
        freq_shifted = tf.signal.fftshift(freq)
        magnitude = tf.abs(freq_shifted)
        magnitude = tf.math.log1p(magnitude)
        magnitude = tf.image.per_image_standardization(magnitude)
        magnitude = tf.image.resize(magnitude, [inputs.shape[1], inputs.shape[2]])
        return magnitude

# Convolutional Transformer Encoder (CTE) Branch
class ConvolutionalTransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, num_layers, num_heads, ff_dim):
        super(ConvolutionalTransformerEncoder, self).__init__()
        self.embedding = layers.Conv2D(embed_dim, kernel_size=3, strides=1, padding='same')
        self.pooling = layers.GlobalAveragePooling2D()
        self.transformer_layers = [
            TransformerBlock(embed_dim, num_heads, ff_dim)
            for _ in range(num_layers)
        ]
        self.fc = layers.Dense(embed_dim)
    
    def call(self, x, training=False):
        x = self.embedding(x)
        x = self.pooling(x)
        x = tf.expand_dims(x, axis=1)
        for transformer_layer in self.transformer_layers:
            x = transformer_layer(x, training=training)
        x = tf.reduce_mean(x, axis=1)
        x = self.fc(x)
        return x

# Language Transformer-Based Encoder
class LanguageTransformerEncoder(layers.Layer):
    def __init__(self, vocab_size, embed_dim, num_layers, num_heads, ff_dim):
        super(LanguageTransformerEncoder, self).__init__()
        self.embedding = layers.Embedding(vocab_size, embed_dim)
        self.transformer_layers = [
            TransformerBlock(embed_dim, num_heads, ff_dim)
            for _ in range(num_layers)
        ]
        self.fc = layers.Dense(embed_dim)
    
    def call(self, x, training=False):
        x = self.embedding(x)
        for transformer_layer in self.transformer_layers:
            x = transformer_layer(x, training=training)
        x = tf.reduce_mean(x, axis=1)
        x = self.fc(x)
        return x

# Weighted Feature Embeddings Fusion
class WeightedFeatureFusion(layers.Layer):
    def __init__(self, embed_dim):
        super(WeightedFeatureFusion, self).__init__()
        self.weight = self.add_weight(shape=(embed_dim,), initializer='ones', trainable=True)
    
    def call(self, feature1, feature2):
        fused = feature1 * self.weight + feature2 * (1 - self.weight)
        return fused

class SVFTDecoder(layers.Layer):
    def __init__(self, embed_dim, num_classes):
        super(SVFTDecoder, self).__init__()
        self.fc = layers.Dense(num_classes)
        # Change this if num_classes is 1 for binary classification
        self.activation = layers.Activation('sigmoid') if num_classes == 1 else layers.Activation('softmax')
    
    def call(self, x, training=False):
        x = self.fc(x)
        x = self.activation(x)
        return x

class DeepfakeDetectionModel(tf.keras.Model):
    def __init__(self, vocab_size, embed_dim, num_layers, num_heads, ff_dim, num_classes, **kwargs):
        super(DeepfakeDetectionModel, self).__init__(**kwargs)
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        self.num_layers = num_layers
        self.num_heads = num_heads
        self.ff_dim = ff_dim
        self.num_classes = num_classes

        self.image_model = ConvolutionalTransformerEncoder(embed_dim, num_layers, num_heads, ff_dim)
        self.text_model = LanguageTransformerEncoder(vocab_size, embed_dim, num_layers, num_heads, ff_dim)
        self.fusion = WeightedFeatureFusion(embed_dim)
        self.decoder = SVFTDecoder(embed_dim, num_classes)

    def call(self, inputs, training=False):
        images, texts = inputs
        img_features = self.image_model(images, training=training)
        text_features = self.text_model(texts, training=training)
        fused_features = self.fusion(img_features, text_features)
        output = self.decoder(fused_features, training=training)
        return output

    def get_config(self):
        config = super(DeepfakeDetectionModel, self).get_config()
        config.update({
            "vocab_size": self.vocab_size,
            "embed_dim": self.embed_dim,
            "num_layers": self.num_layers,
            "num_heads": self.num_heads,
            "ff_dim": self.ff_dim,
            "num_classes": self.num_classes,
        })
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)

# Training script
if __name__ == "__main__":
    def load_image_text_dataset(image_dir, metadata_file, batch_size=8, img_size=(224, 224), max_seq_length=50):
        # Load images from directory
        image_dataset = image_dataset_from_directory(
            image_dir,
            labels=None,
            image_size=img_size,
            batch_size=batch_size,
            shuffle=False  # Shuffle will be handled later
        )

        # Load metadata from CSV file
        metadata = pd.read_csv(metadata_file)

        # Extract texts and labels from metadata
        texts = metadata['Name'].tolist()
        labels = metadata['Label'].tolist()

        # Convert string labels to integers
        label_mapping = {'Real': 0, 'Fake': 1}
        int_labels = [label_mapping[label] for label in labels]

        # Tokenize and pad text sequences
        tokenizer.fit_on_texts(texts)
        sequences = tokenizer.texts_to_sequences(texts)
        text_tensor = pad_sequences(sequences, maxlen=max_seq_length)
        label_tensor = tf.constant(int_labels)

        # Create a dataset by zipping images, texts, and labels
        # Ensure that image_dataset has the same order as metadata
        image_iterator = image_dataset.as_numpy_iterator()
        def gen():
            for img_batch, _ in image_iterator:
                for i in range(len(img_batch)):
                    img = img_batch[i]
                    text = text_tensor[i]
                    label = label_tensor[i]
                    yield (img, text), label


        dataset = tf.data.Dataset.from_generator(
            gen,
            output_signature=(
                (
                    tf.TensorSpec(shape=(img_size[0], img_size[1], 3), dtype=tf.float32),
                    tf.TensorSpec(shape=(max_seq_length,), dtype=tf.int32)
                ),
                tf.TensorSpec(shape=(), dtype=tf.int32)
            )
        )
        dataset = dataset.shuffle(buffer_size=1000).batch(batch_size).prefetch(tf.data.AUTOTUNE)
        return dataset

    # Load dataset
    image_directory = "C:\\Users\\Lord Sen\\ML\\DeepFake_det\\FF++_Faces_224\\FF++_Faces_224"  # Update as needed
    metadata_file = "C:\\Users\\Lord Sen\\ML\\metadata.csv"  # Update as needed
    train_dataset = load_image_text_dataset(image_directory, metadata_file)
    
    # Define model hyperparameters
    vocab_size = 10000
    embed_dim = 128
    num_layers = 2
    num_heads = 4
    ff_dim = 256
    num_classes = 1

    # Initialize the model
    model = DeepfakeDetectionModel(
        vocab_size=vocab_size, 
        embed_dim=embed_dim, 
        num_layers=num_layers, 
        num_heads=num_heads, 
        ff_dim=ff_dim, 
        num_classes=num_classes
    )

    # Build the model by passing a sample input
    sample_images = tf.random.uniform((1, 224, 224, 3))
    sample_texts = tf.random.uniform((1, 50), maxval=vocab_size, dtype=tf.int32)  # Fix: Specify maxval
    model((sample_images, sample_texts), training=True)

    # Compile the model
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), 
        loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), 
        metrics=['accuracy']
    )

    # Train the model
    #model.fit(train_dataset, epochs=10)

    # Save the model in the `.keras` format
    model.save('deepfake_detection_model.keras')

    # Define custom objects for loading the model
    custom_objects = {
       "DeepfakeDetectionModel": DeepfakeDetectionModel, 
       "TransformerBlock": TransformerBlock,
       "ConvolutionalTransformerEncoder": ConvolutionalTransformerEncoder,
       "LanguageTransformerEncoder": LanguageTransformerEncoder,
       "WeightedFeatureFusion": WeightedFeatureFusion,
       "SVFTDecoder": SVFTDecoder
   }

    # Load the model
    loaded_model = tf.keras.models.load_model('deepfake_detection_model.keras', custom_objects=custom_objects)


Found 400 files.


C:\Users\Lord Sen\anaconda3\Lib\site-packages\keras\src\ops\nn.py:545: UserWarning: You are using a softmax over axis 3 of a tensor of shape (1, 4, 1, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(
C:\Users\Lord Sen\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'adam', because it has 150 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [5]:
# Example input data
images = tf.random.normal([8, 224, 224, 3])  # Batch of 8 images
texts = tf.random.uniform([8, 50], maxval=vocab_size, dtype=tf.int32)  # Batch of 8 text sequences

    # Forward pass
outputs = model((images, texts))
print(outputs)  # Should be [8, num_classes]

tf.Tensor(
[[-0.58494735  0.6527562 ]
 [-1.0875294   0.91202134]
 [ 0.1769378   1.0657979 ]
 [-0.23638088  1.3789895 ]
 [-0.2632763   0.55552715]
 [-0.10037638  0.5711725 ]
 [-0.8815618   1.5987645 ]
 [ 0.29555595  1.0402079 ]], shape=(8, 2), dtype=float32)


C:\Users\Lord Sen\anaconda3\Lib\site-packages\keras\src\ops\nn.py:545: UserWarning: You are using a softmax over axis 3 of a tensor of shape (8, 4, 1, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


In [6]:
import tensorflow as tf
import numpy as np

model = DeepfakeDetectionModel(vocab_size=vocab_size, embed_dim=embed_dim, num_layers=num_layers, num_heads=num_heads, ff_dim=ff_dim, num_classes=num_classes)

# Example input data
images = tf.random.normal([8, 224, 224, 3])  # Batch of 8 images
texts = tf.random.uniform([8, 50], maxval=vocab_size, dtype=tf.int32)  # Batch of 8 text sequences

# Forward pass
outputs = model((images, texts))

# Apply softmax to get probabilities
probabilities = tf.nn.softmax(outputs, axis=1)

# Get the class with the highest probability
predictions = tf.argmax(probabilities, axis=1)

# Convert predictions to numpy array for easier manipulation
predictions = predictions.numpy()

# Print results
for i, prediction in enumerate(predictions):
    if prediction == 0:
        print(f"Image {i} is classified as Real")
    else:
        print(f"Image {i} is classified as Fake")


Image 0 is classified as Fake
Image 1 is classified as Real
Image 2 is classified as Fake
Image 3 is classified as Fake
Image 4 is classified as Fake
Image 5 is classified as Fake
Image 6 is classified as Real
Image 7 is classified as Real


In [16]:
import tensorflow as tf
import numpy as np
import gradio as gr
from PIL import Image

# Assuming the model class definitions are already provided as above

# Function to preprocess the image
def preprocess_image(image):
    image = image.resize((224, 224))
    image = np.array(image) / 255.0
    image = np.expand_dims(image, axis=0)
    return image

# Function to classify the image
def classify_image(image):
    # Preprocess the image
    image = preprocess_image(image)
    
    # Example text data (this should be adapted to your specific case)
    vocab_size = 10000  # Example vocab size
    texts = tf.random.uniform([1, 50], maxval=vocab_size, dtype=tf.int32)
    
    # Forward pass
    outputs = model((image, texts))
    
    # Apply softmax to get probabilities
    probabilities = tf.nn.softmax(outputs, axis=1)
    
    # Get the class with the highest probability
    prediction = tf.argmax(probabilities, axis=1).numpy()[0]
    
    if prediction == 1:
        return "Real"
    else:
        return "Fake"

# Define the model parameters
vocab_size = 10000
embed_dim = 128
num_layers = 2
num_heads = 4
ff_dim = 256
num_classes = 2

# Initialize the model
model = DeepfakeDetectionModel(vocab_size=vocab_size, embed_dim=embed_dim, num_layers=num_layers, num_heads=num_heads, ff_dim=ff_dim, num_classes=num_classes)

# Define the Gradio interface
iface = gr.Interface(
    fn=classify_image,
    inputs=gr.Image(type="pil"),
    outputs="text",
    title="Deepfake Detection_",
    description="Upload an image to check if it's real or fake."
)

# Launch the Gradio interface
iface.launch()

Running on local URL:  http://127.0.0.1:7869

To create a public link, set `share=True` in `launch()`.


In [17]:
import tensorflow as tf
import numpy as np
import gradio as gr
from PIL import Image

# Assuming the model class definitions are already provided as above

# Function to preprocess the image
def preprocess_image(image):
    image = image.resize((224, 224))
    image = np.array(image) / 255.0
    image = np.expand_dims(image, axis=0)
    return image

# Function to classify the image
def classify_image(image):
    # Preprocess the image
    image = preprocess_image(image)
    
    # Example text data (this should be adapted to your specific case)
    vocab_size = 10000  # Example vocab size
    texts = tf.random.uniform([1, 50], maxval=vocab_size, dtype=tf.int32)
    
    # Forward pass
    outputs = model((image, texts))
    
    # Apply softmax to get probabilities
    probabilities = tf.nn.softmax(outputs, axis=1)
    
    # Get the class with the highest probability
    prediction = tf.argmax(probabilities, axis=1).numpy()[0]
    
    if prediction == 0:
        return "Real"
    else:
        return "Fake"

# Define the model parameters
vocab_size = 10000
embed_dim = 128
num_layers = 2
num_heads = 4
ff_dim = 256
num_classes = 2

# Initialize the model
model = DeepfakeDetectionModel(vocab_size=vocab_size, embed_dim=embed_dim, num_layers=num_layers, num_heads=num_heads, ff_dim=ff_dim, num_classes=num_classes)

# Define the Gradio interface
iface = gr.Interface(
    fn=classify_image,
    inputs=gr.Image(type="pil"),
    outputs="text",
    title="Deepfake Detection_",
    description="Upload an image to check if it's real or fake."
)

# Launch the Gradio interface
iface.launch()

Running on local URL:  http://127.0.0.1:7870

To create a public link, set `share=True` in `launch()`.


C:\Users\Lord Sen\anaconda3\Lib\site-packages\keras\src\ops\nn.py:545: UserWarning: You are using a softmax over axis 3 of a tensor of shape (1, 4, 1, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(
C:\Users\Lord Sen\anaconda3\Lib\site-packages\keras\src\ops\nn.py:545: UserWarning: You are using a softmax over axis 3 of a tensor of shape (1, 4, 1, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(
C:\Users\Lord Sen\anaconda3\Lib\site-packages\keras\src\ops\nn.py:545: UserWarning: You are using a softmax over axis 3 of a tensor of shape (1, 4, 1, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(
C:\Users\Lord Sen\anac